### 소방청 - 경기도 소방기관 추출

In [1]:
import time 
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.2967198&myY=127.0078127&searchKeyword='

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
# driver = webdriver.Chrome(r'C:\Users\user\Downloads\chromedriver.exe')  # 집에서
driver.get(url)

In [3]:
# 검색 입력상자 찾아서 경기도 입력하기
driver.find_element(By.ID, 'searchKeyword').send_keys('경기도')

In [4]:
# 검색 버튼 찾아서 클릭하기
driver.find_element(By.ID, 'fsSearchBtn').click()

In [5]:
#페이지 이동해서 HTML 코드를 BeautifulSoup으로 parsing
soup =BeautifulSoup(driver.page_source,'html.parser')
lis = soup.select('.stations-list > li')
len(lis) # 10건

10

In [6]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
name

'군포소방서-송정-119 안전센터'

In [7]:
addr = li.select_one('address').get_text().strip()
addr

'경기도 군포시 산본동1156'

In [8]:
tel = li.select_one('.tel').get_text().strip()
tel

'031-479-8503'

In [12]:
lines = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.select_one('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append({'관서명':name, '주소':addr, '전화번호':tel})
df = pd.DataFrame(lines)
df

,관서명,주소,전화번호
0,군포소방서-송정-119 안전센터,경기도 군포시 산본동1156,031-479-8503
1,부천소방서-원종-119 안전센터,경기도 부천시 원종로79번길 41 (원종동),032-650-4554
2,부천소방서-괴안-119 안전센터,경기도 부천시 부광로8번길 49 (괴안동),032-650-4544
3,부천소방서-중동-119 안전센터,경기도 부천시 부일로 346 (중동),032-650-4534
4,부천소방서-내동-119 안전센터,경기도 부천시 신흥로 364 (내동),032-650-4524
5,부천소방서-중앙-119 안전센터,경기도 부천시 신흥로 115 (중동),032-650-4504
6,분당소방서-구미-119 안전센터,경기도 성남시 분당구 돌마로 138 (구미동),031-8018-3551
7,분당소방서-야탑-119 안전센터,경기도 성남시 분당구 양현로 485 (야탑4동),031-8018-3531
8,분당소방서-수내-119 안전센터,경기도 성남시 분당구 백현로 241 (수내동),031-8018-3521
9,분당소방서-판교-119 안전센터,경기도 성남시 분당구 대왕판교로644번길 62 (삼평동),031-8018-3511


- 두번째 페이지로 이동

In [13]:
# copy xpath
# //*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

- next(>) page로 이동

In [14]:
driver.find_element(By.CLASS_NAME, 'next_page').click()

- 모든 페이지에 대해서 적용(단, 이동은 selenium으로)

In [15]:
# 첫번째 페이지
driver.get(url)
time.sleep(1) # 페이지 이동시 필요
driver.find_element(By.ID, 'searchKeyword').send_keys('경기도')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [17]:
# 검색결과 건수 확인
count = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/div/p/strong[2]').text
count = int(count[:-1])
count

100

In [ ]:
# 100건이라 10번인거는
# 한페이지에 10건이 있어서
# 100건이 있으니까 10 # i는 0~10 0은 skip,1은 두번째페이지,2는 넥스트,3은 두번째,4는 넥스트

In [18]:
import math
total_page = int(math.ceil(count / 10)) # 100/10 = 10
total_page

10

In [19]:
lines = []
for i in range(total_page): # 100건이 있으니까 10 # i는 0~10 0은 skip,1은 두번째페이지,2는 넥스트,3은 두번째,4는 넥스트
    if i % 2 == 1: # 페이지이동 (1,3,5,.. 두번째페이지 코드)
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)
    if i > 0 and i % 2 == 0:  # 페이지이동 (0은 skip) and (2,4,6,.. next 코드)
        driver.find_element(By.CLASS_NAME, 'next_page').click()
        time.sleep(2)

    # 페이지 이동후 페이지 소스 가져오기
    soup =BeautifulSoup(driver.page_source,'html.parser')  # 이동한 페이지 소스
    lis = soup.select('.stations-list > li')    # 이동한 페이지소스에서 li선택
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.select_one('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append({'관서명':name, '주소':addr, '전화번호':tel})


In [21]:
driver.close()

In [20]:
df = pd.DataFrame(lines)
df.tail()

,관서명,주소,전화번호
95,의정부소방서-금오-119 안전센터,경기도 의정부시 호국로 1434-14,031-849-7510
96,의정부소방서-중앙-119 안전센터,경기도 의정부시 의정로 48,031-849-7500
97,일산소방서-대화-119 안전센터,경기도 고양시 일산서구 대화동 2321-2,031-930-0572
98,일산소방서-고봉-119 안전센터,경기도 고양시 일산동구 성현로 429,031-930-0551
99,일산소방서-중산-119 안전센터,경기도 고양시 일산동구 중산로 218,031-930-0541


In [30]:
df.head(90).tail(10)

,관서명,주소,전화번호
80,파주소방서-월롱-119 안전센터,경기도 파주시 월롱면 엘씨디로 310,031-956-9551
81,파주소방서-탄현-119 안전센터,경기도 파주시 탄현면 필승로 458,031-956-9541
82,파주소방서-광탄-119 안전센터,경기도 파주시 광탄면 혜음로 1040,031-956-9531
83,파주소방서-법원-119 안전센터,경기도 파주시 법원읍 화합로 141,031-956-9521
84,파주소방서-금촌-119 안전센터,경기도 파주시 가나무로 136,031-956-9511
85,남양주소방서-가운-119 안전센터,경기도 남양주시 가운로 16,031-590-0572
86,남양주소방서-오남-119 안전센터,경기도 남양주시 오남읍 진건오남로 772,031-590-0592
87,남양주소방서-별내-119 안전센터,경기도 남양주시 송산로 41,031-590-0562
88,남양주소방서-와부-119 안전센터,경기도 남양주시 와부읍 덕소로 97번길 34-2,031-590-0532
89,남양주소방서-화도-119 안전센터,경기도 남양주시 화도읍 수레로 1260번길 25,031-590-0542
